In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column,Integer,String,Numeric,Text,Float,ForeignKey
from sqlalchemy.orm import sessionmaker,relationship
import datetime
import dateutil.relativedelta
import quandl

In [2]:
engine = create_engine("sqlite:///pub_atx.sqlite")

In [3]:
Base = declarative_base()

In [4]:
class Company(Base):
    __tablename__ = 'cmpny'
    
    id_cmpny = Column(Integer, primary_key=True)
    rank = Column(Integer)
    name = Column(Text)
    address = Column(Text)
    city = Column(Text)
    state = Column(Text)
    zip_code = Column(Text)
    phn_nbr = Column(Text)
    website = Column(Text)
    tckr = Column(Text)
    ipo_yr = Column(Integer)
    yr_founded = Column(Integer)
    gen_buss_desc = Column(Text)
    curr_top_exec = Column(Text)
    yr_estblsh = Column(Integer)
    austin_staff_cnt = Column(Float)
    comp_staff_cnt = Column(Float)
    tot_local_emp_cnt = Column(Float)
    exchng = Column(Text)
    lat = Column(Float)
    lng = Column(Float)
    industry = Column(Text)
    sector = Column(Text)
    marketcap = Column(Float)
    
    def __repr__(self):
        return f"id={self.id},name=[self.name]"
    
class CompanyPrcsMnthly(Base):
    __tablename__ = 'cmpny_prcs_mnthly'
    
    id_cmpny_prcs_mnthly = Column(Integer, primary_key=True)
    id_cmpny = Column(Integer)
    date = Column(Text)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Float)
    
    
    def __repr__(self):
        return f"id={self.id_cmpny_prcs_mnthly},name=[self.id_cmpny]"
    
class CompanyPrcsDaily(Base):
    __tablename__ = 'cmpny_prcs_daily'
    
    id_cmpny_prcs_daily = Column(Integer, primary_key=True)
    id_cmpny = Column(Integer)
    date = Column(Text)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Float)
    
    
    def __repr__(self):
        return f"id={self.id_cmpny_prcs_daily},name=[self.id_cmpny]"

In [5]:
!rm pub_atx.sqlite

In [6]:
Base.metadata.create_all(engine)

In [7]:
engine.table_names()

['cmpny', 'cmpny_prcs_daily', 'cmpny_prcs_mnthly']

In [8]:
conn =engine.connect()
df_company = pd.read_csv("clean_company_list.csv")
data = df_company.to_dict(orient='records')
conn.execute(Company.__table__.delete())
conn.execute(Company.__table__.insert(),data)

In [9]:
companies = engine.execute("select * from cmpny").fetchall()

In [10]:
companies

[(1, 1, 'Whole Foods Market Inc.', '550 Bowie St.', 'Austin', 'TX', '78703', '512-476-1206', 'wholefoodsmarket.com', 'WFMI', 1992, 1980, 'Retail grocery of natural and organic foods; operates 431 stores:U.S. (412), Canada (10), U.K.(9)', 'John  Mackey,  CEO', 1980, 3000.0, 87000.0, 2530.0, 'Nasdaq', 30.270634700000002, -97.7528921, 'unknown', 'unknown', 0.0),
 (2, 2, 'National Instruments Corp.', '11500 N. MoPac Expy.', 'Austin', 'TX', '78759', '512-683-0100', 'ni.com', 'NATI', 1995, 1976, 'Designs, manufactures and sells tools to engineers and scientists, equipping them with systems that accelerate productivity, innovation and discovery ', 'Alex  Davern,  President/CEO', 1976, 2500.0, 7552.0, 2600.0, 'Nasdaq', 30.4083821, -97.7262444, 'Application Software', 'Technology', 6679636113.0),
 (3, 3, 'Cirrus Logic Inc.', '800 W. Sixth St.', 'Austin', 'TX', '78701', '512-851-4000', 'cirrus.com', 'CRUS', 1989, 1984, 'Semiconductor developing high-precision, analog and mixed-signal integrated 

In [11]:

d1 = datetime.date.today()
d2 = d1 - dateutil.relativedelta.relativedelta(months=36)
start_dt = d2.strftime("%Y-%m-%d")
end_dt = d1.strftime("%Y-%m-%d")


In [12]:
quandl.ApiConfig.api_key = 


In [13]:
conn.execute(CompanyPrcsMnthly.__table__.delete())

In [14]:
for company in companies:
    try:
       
        data = quandl.get("WIKI" +"/" + company.tckr,collapse="monthly", start_date=start_dt, end_date=end_dt)
        data = data.reset_index()
        data = data[['Date','Open','High','Low','Close','Volume']]
        data.columns = ["date","open","high","low","close","volume"]
        data["id_cmpny"] = company.id_cmpny
        data = data[["id_cmpny", "date","open","high","low","close","volume"]]
        data['date'] = data['date'].astype('str')
        data = data.to_dict(orient='records')
        
        conn.execute(CompanyPrcsMnthly.__table__.insert(),data)
    except quandl.NotFoundError as e:
        print(company.tckr + " not found ")
        continue

WFMI not found 
BETR not found 
USAC not found 
XPLR not found 
APPS not found 
STRS not found 
UPLD not found 
ASUR not found 
AGLE not found 
FPP not found 
ASTC not found 
VRML not found 
IPWR not found 
CRDS not found 
SVRA not found 
VYEY not found 
SCON not found 
XBIT not found 
MIRN not found 


In [15]:
conn.execute(CompanyPrcsDaily.__table__.delete())

In [16]:
for company in companies:
    try:
       
        data = quandl.get("WIKI" +"/" + company.tckr, start_date=start_dt, end_date=end_dt)
        data = data.reset_index()
        data = data[['Date','Open','High','Low','Close','Volume']]
        data.columns = ["date","open","high","low","close","volume"]
        data["id_cmpny"] = company.id_cmpny
        data = data[["id_cmpny", "date","open","high","low","close","volume"]]
        data['date'] = data['date'].astype('str')
        data = data.to_dict(orient='records')
        
        conn.execute(CompanyPrcsDaily.__table__.insert(),data)
    except quandl.NotFoundError as e:
        print(company.tckr + " not found ")
        continue

WFMI not found 
BETR not found 
USAC not found 
XPLR not found 
APPS not found 
STRS not found 
UPLD not found 
ASUR not found 
AGLE not found 
FPP not found 
ASTC not found 
VRML not found 
IPWR not found 
CRDS not found 
SVRA not found 
VYEY not found 
SCON not found 
XBIT not found 
MIRN not found 


In [17]:

engine.execute("select count(*) from cmpny_prcs_mnthly").fetchall()

[(591,)]

In [18]:
engine.execute("select count(*) from cmpny_prcs_daily").fetchall()

[(12046,)]

In [19]:

# engine.execute("select * from cmpny_prcs_mnthly").fetchall()

In [20]:
# engine.execute("select * from cmpny_prcs_daily").fetchall()